In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlcontext = SQLContext(sc)

In [2]:
path = "hdfs://wolf.analytics.private/user/slx4192/data/crime/Crimes_-_2001_to_present.csv"
mydata = sqlcontext.read.csv(path, header = True)

In [3]:
mydata.show(1)

+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|                Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|01/01/2001 11:00:...|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        

In [4]:
# # sample a fraction of data
# mydata.sample(.0001, False, 42).collect()
# mydata.head(1)

## Question 1
By using SparkSQL, generate a histogram of average crime events by month. Find an explanation of results. (10 pts)

In [5]:
mydata.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'string'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'string'),
 ('Beat', 'string'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'string'),
 ('Y Coordinate', 'string'),
 ('Year', 'string'),
 ('Updated On', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Location', 'string')]

ID
Case Number
Date='09/03/2017 10:00:00 PM'
Block='030XX N CENTRAL AVE'
IUCR='0915'
Primary Type='MOTOR VEHICLE THEFT'
Description='TRUCK, BUS, MOTOR HOME'
Location Description='VEHICLE NON-COMMERCIAL'
Arrest='false'
Domestic='false'
Beat='2514'
District='025'
Ward='31'
Community Area='19'
FBI Code='07'
X Coordinate='1138505'
Y Coordinate='1919683'
Year='2017'
Updated On='02/10/2018 03:50:01 PM'
Latitude='41.935738494'
Longitude='-87.766399726'
Location='(41.935738494, -87.766399726)')

In [6]:
mydata_month = mydata.withColumn('Month', mydata['Date'].substr(0, 2))

In [7]:
mydata_month.groupBy("Month","Year").count().show()

+-----+----+-----+
|Month|Year|count|
+-----+----+-----+
|   04|2012|27182|
|   04|2020|10537|
|   03|2014|22188|
|   07|2004|43238|
|   06|2004|41134|
|   07|2016|24837|
|   02|2005|31987|
|   06|2016|24045|
|   07|2012|31979|
|   06|2014|25430|
|   05|2003|41195|
|   01|2004|35106|
|   03|2017|20535|
|   08|2014|25879|
|   03|2009|33688|
|   04|2019|20941|
|   04|2007|35634|
|   04|2006|36476|
|   04|2003|39789|
|   02|2009|28234|
+-----+----+-----+
only showing top 20 rows



In [8]:
monthly_avg = mydata_month.groupBy("Month","Year").count().select("Month","count").groupBy("Month").avg().orderBy("Month")

In [9]:
monthly_avg.show()

+-----+------------------+
|Month|        avg(count)|
+-----+------------------+
|   01|           28404.2|
|   02|          25014.85|
|   03|           29715.8|
|   04|           29541.4|
|   05| 32995.73684210526|
|   06| 32839.52631578947|
|   07| 34506.05263157895|
|   08| 34159.15789473684|
|   09|32008.894736842107|
|   10|32353.894736842107|
|   11|29134.894736842107|
|   12|27658.473684210527|
+-----+------------------+



In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
monthly_avg.toPandas().hist(column='avg(count)',bins=20)